### **Multilayer Perceptron - PyTorch**


#### [Step 1] Dataset & DataLoader

- **Dataset** : Dataset 클래스를 상속 --> Custom Instance를 생성

- **DataLoader**: 학습 시 각 Instance에 쉽게 접근하기 위해 순회 가능한 객체(iterable) 생성, shuffling 및 batch 등의 작업 수행

<p align="center">
<img src = https://sebastianraschka.com/images/blog/2022/datapipes/loader-flow.png width = 500>
</p>

<p align="center">
<img src = https://sebastianraschka.com/images/blog/2022/datapipes/dataflow-good.png width = 350>
</p>

- **Sample code**
```
from torch.utils.data import  TensorDataset, DataLoader
```
```
# X,y로 분할한 데이터를 tensor로 변환
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.int64)
y_test = torch.tensor(y_test, dtype=torch.int64)
```
```
# tensor를 TensorDataset으로 생성 - X와 y가 짝으로 이루어짐
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
```
```
# DataLoader 형태로 생성
train_dataloader = DataLoader(train_dataset, batch_size=10, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=10, shuffle=True)
```


#### [Step 2] 디바이스 설정

- 일반적으로 인공신경망의 학습은 (가능하다면) GPU를 사용하는 것이 바람직함
    - Colab Runtime 설정 변경
- GPU를 사용하여 학습을 진행하도록 명시적으로 작성 필요
- 연산 유형에 따라 GPU에서 수행이 불가능한 경우도 존재하는데, 그럴 경우도 마찬가지로 명시적으로 어떤 프로세서에서 연산을 수행해야하는지 코드로 작성해야함 

```
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = NeuralNetwork().to(device)
```


#### [Step 3] 신경망 모델 생성

- **torch.nn 패키지**는 신경망 생성 및 학습 시 설정해야하는 다양한 기능을 제공

```
import torch.nn as nn
```
- 신경망을 **nn.Module**을 상속받아 정의
    - __ __init__ __(): 신경망에서 사용할 layer를 초기화하는 부분
    - __forward()__: feed foward 연산 수행 시, 각 layer의 입출력이 어떻게 연결되는지를 지정

```
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.input_layer    = nn.Linear(4, 16)
        self.hidden_layer1  = nn.Linear(16, 32)
        self.output_layer   = nn.Linear(32, 3)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out =  self.relu(self.input_layer(x))
        out =  self.relu(self.hidden_layer1(out))
        out =  self.output_layer(out)
        return out

```


#### [Step 4] 네트워크 컴파일(Compiling)

- 학습 시 필요한 정보들(loss function, optimizer)을 선언
- 일반적으로 loss와 optimizer는 아래와 같이 변수로 선언하고, 변수를 train/test 시 참고할 수 있도록 매개변수로 지정해줌 

```
learning_rate = 0.01
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)
```


#### [Step 5] 네트워크 학습(Training)

- **신경망의 학습과정**을 별도의 함수로 구성하는 것이 일반적
    - feed forward -> loss -> error back propagation -> (log) -> (반복)

```
def train_loop(train_loader, model, loss_fn, optimizer):
    for batch, (X, y) in enumerate(train_loader):
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
```

#### [Step 6] 학습된 네트워크 테스트

- 학습과정과 비슷하나 error back propagate하는 부분이 없음
    - feed forward -> loss ->  (log) -> (반복)

```
def test_loop(test_loader, model, loss_fn):
    size = len(test_loader.dataset)
    num_batches = len(test_loader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in test_loader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:8f}\n")


#### [Step 7] Iteration
- 신경망 학습은 여러 epochs을 반복해서 수행하면서 모델을 구성하는 최적의 파라미터를 찾음
- 지정한 epochs 수만큼 **학습**과정과 **평가**과정을 반복하면서, 모델의 성능(loss, accuracy 등)을 체크함

```
epochs = 10
for i in range(epochs) :
    print(f"Epoch {i+1} \n------------------------")
    train_loop(train_dataloader, model, loss, optimizer)
    test_loop(test_dataloader, model, loss)
print("Done!")
```

### **MLP - IRIS 데이터셋**

데이터설명 : 아이리스(붓꽃) 데이터에 대한 데이터이다. 꽃잎의 각 부분의 너비와 길이등을 측정한 데이터이며 150개의 레코드로 구성되어 있다. 아이리스 꽃은 아래의 그림과 같다. 프랑스의 국화라고 한다.

- 데이터명 : IRIS (아이리스, 붗꽃 데이터)  
- 레코드수 : 150개
- 필드개수 : 5개(총 6개의 필드로 구성, caseno는 순서를 표시하므로 분석에서 제외), 2번째부터 5번째의 4개의 필드는 입력 변수로 사용되고, 맨 아래의 Species 속성이 목표(종속) 변수로 사용된다.

  - caseno	일련번호이다. (1부터 150까지 입력된다.)
  - Sepal Length	꽃받침의 길이 정보이다.
  - Sepal Width	꽃받침의 너비 정보이다.
  - Petal Length	꽃잎의 길이 정보이다.
  - Petal Width	꽃잎의 너비 정보이다.  
  - Species	꽃의 종류 정보이다.  setosa / versicolor / virginica 의 3종류로 구분된다.

#### [Step 01] Load libraries & Datasets



In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import  TensorDataset, DataLoader

# 데이터 불러오기
iris = load_iris()
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)



In [17]:
print(df.shape)
df.describe()


(150, 4)


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333
std,0.828066,0.435866,1.765298,0.762238
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [18]:
df['label'] = iris.target

# 데이터분할
y = df['label']
X = df.drop(['label'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, random_state=42, stratify=y)

#### [Step2] Create DataLoader

In [19]:
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.int64)
y_test = torch.tensor(y_test, dtype=torch.int64)

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_dataloader = DataLoader(train_dataset, batch_size=10, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=10, shuffle=True)

#### [Step3] Set Network Structure

In [20]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.input_layer    = nn.Linear(4, 16)
        self.hidden_layer1  = nn.Linear(16, 32)
        self.output_layer   = nn.Linear(32, 3)
        self.relu = nn.ReLU()    
    
    def forward(self,x):
        out =  self.relu(self.input_layer(x))
        out =  self.relu(self.hidden_layer1(out))
        out =  self.output_layer(out)
        return out

#### [Step4] Create Model instance

In [21]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'device = {device}')

model = NeuralNetwork().to(device)


device = cuda


#### [Step5] Model compile

In [22]:
# 모델 컴파일
learning_rate = 0.001
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

#### [Step6] Set train loop

In [23]:
def train_loop(train_loader, model, loss_fn, optimizer):
    size = len(train_loader.dataset)

    for batch, (X, y) in enumerate(train_loader):
        X, y = X.to(device), y.to(device)
        pred = model(X)

        # 손실 계산
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        loss, current = loss.item(), batch * len(X)
        print(f'loss: {loss:>7f}  [{current:>5d}]/{size:5d}')


#### [Step7] Set test loop

In [24]:
def test_loop(test_loader, model, loss_fn):
    size = len(test_loader.dataset)
    num_batches = len(test_loader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in test_loader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:8f}\n")

#### [Step8] Run model

In [26]:
# 모델 실행
epochs = 100

for i in range(epochs) :
    print(f"Epoch {i+1} \n------------------------")
    train_loop(train_dataloader, model, loss, optimizer)
    test_loop(test_dataloader, model, loss)
print("Done!")

Epoch 1 
------------------------
loss: 0.452185  [    0]/  112
loss: 0.612783  [   10]/  112
loss: 0.301838  [   20]/  112
loss: 0.385962  [   30]/  112
loss: 0.694239  [   40]/  112
loss: 0.585045  [   50]/  112
loss: 0.595845  [   60]/  112
loss: 0.393974  [   70]/  112
loss: 0.408531  [   80]/  112
loss: 0.483114  [   90]/  112
loss: 0.474614  [  100]/  112
loss: 0.501538  [   22]/  112
Test Error: 
 Accuracy: 76.3%, Avg loss: 0.495204

Epoch 2 
------------------------
loss: 0.507666  [    0]/  112
loss: 0.529120  [   10]/  112
loss: 0.520601  [   20]/  112
loss: 0.389667  [   30]/  112
loss: 0.238824  [   40]/  112
loss: 0.366642  [   50]/  112
loss: 0.590740  [   60]/  112
loss: 0.580313  [   70]/  112
loss: 0.527533  [   80]/  112
loss: 0.377504  [   90]/  112
loss: 0.539492  [  100]/  112
loss: 0.492341  [   22]/  112
Test Error: 
 Accuracy: 81.6%, Avg loss: 0.482780

Epoch 3 
------------------------
loss: 0.569176  [    0]/  112
loss: 0.341312  [   10]/  112
loss: 0.350611  